In [15]:
from urllib.request import urlopen
from zipfile import ZipFile
import io
import shapefile
import geopandas as gpd
from shapely.geometry import shape  
import pandas as pd
import requests
import matplotlib.pyplot as plt

In [20]:
#1 reading in American Community Survey Data
Education_Income = pd.read_csv("Manhattan_Education_Income.csv")
Education_Income

,GEO_ID,NAME,S1501_C01_001E,S1501_C01_001M,S1501_C01_002E,S1501_C01_002M,S1501_C01_003E,S1501_C01_003M,S1501_C01_004E,S1501_C01_004M,...,S1501_C06_060E,S1501_C06_060M,S1501_C06_061E,S1501_C06_061M,S1501_C06_062E,S1501_C06_062M,S1501_C06_063E,S1501_C06_063M,S1501_C06_064E,S1501_C06_064M
0,id,Geographic Area Name,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT...,Margin of Error!!Total!!AGE BY EDUCATIONAL ATT...,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT...,Margin of Error!!Total!!AGE BY EDUCATIONAL ATT...,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT...,Margin of Error!!Total!!AGE BY EDUCATIONAL ATT...,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT...,Margin of Error!!Total!!AGE BY EDUCATIONAL ATT...,...,Estimate!!Percent Female!!MEDIAN EARNINGS IN T...,Margin of Error!!Percent Female!!MEDIAN EARNIN...,Estimate!!Percent Female!!MEDIAN EARNINGS IN T...,Margin of Error!!Percent Female!!MEDIAN EARNIN...,Estimate!!Percent Female!!MEDIAN EARNINGS IN T...,Margin of Error!!Percent Female!!MEDIAN EARNIN...,Estimate!!Percent Female!!MEDIAN EARNINGS IN T...,Margin of Error!!Percent Female!!MEDIAN EARNIN...,Estimate!!Percent Female!!MEDIAN EARNINGS IN T...,Margin of Error!!Percent Female!!MEDIAN EARNIN...
1,1400000US36061000100,"Census Tract 1, New York County, New York",0,12,0,12,0,12,0,12,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
2,1400000US36061000201,"Census Tract 2.01, New York County, New York",249,79,50,41,36,39,110,54,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
3,1400000US36061000202,"Census Tract 2.02, New York County, New York",530,208,125,93,112,69,225,140,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
4,1400000US36061000500,"Census Tract 5, New York County, New York",0,12,0,12,0,12,0,12,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,1400000US36061030900,"Census Tract 309, New York County, New York",988,285,101,91,426,183,372,169,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
285,1400000US36061031100,"Census Tract 311, New York County, New York",0,12,0,12,0,12,0,12,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
286,1400000US36061031703,"Census Tract 317.03, New York County, New York",92,71,0,17,23,30,12,18,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
287,1400000US36061031704,"Census Tract 317.04, New York County, New York",321,185,27,43,18,30,99,86,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)


In [23]:
#2 reading in our Census Tract Data shapefile as a Geopandas.dateframe
gpd2019shp = gpd.read_file("tl_2019_36_tract.shp")
gpd2019shp

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,36,081,044800,36081044800,448,Census Tract 448,G5020,S,208002,0,+40.7110219,-073.8026344,"POLYGON ((-73.80646 40.71206, -73.80556 40.712..."
1,36,081,045800,36081045800,458,Census Tract 458,G5020,S,245281,0,+40.7152626,-073.7909261,"POLYGON ((-73.79364 40.71382, -73.79362 40.713..."
2,36,081,046200,36081046200,462,Census Tract 462,G5020,S,249611,0,+40.7098547,-073.7879749,"POLYGON ((-73.79203 40.71107, -73.79101 40.711..."
3,36,081,046300,36081046300,463,Census Tract 463,G5020,S,159415,0,+40.7440007,-073.8710900,"POLYGON ((-73.87468 40.74335, -73.87423 40.743..."
4,36,081,046400,36081046400,464,Census Tract 464,G5020,S,228767,0,+40.7168637,-073.7869958,"POLYGON ((-73.79187 40.71379, -73.79085 40.714..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,36,081,027900,36081027900,279,Census Tract 279,G5020,S,177155,0,+40.7516729,-073.8802222,"POLYGON ((-73.88189 40.75208, -73.88096 40.752..."
4914,36,081,028000,36081028000,280,Census Tract 280,G5020,S,165398,0,+40.6883158,-073.7797191,"POLYGON ((-73.78243 40.69034, -73.78160 40.690..."
4915,36,081,028100,36081028100,281,Census Tract 281,G5020,S,174444,0,+40.7522203,-073.8825920,"POLYGON ((-73.88431 40.75567, -73.88351 40.755..."
4916,36,081,028200,36081028200,282,Census Tract 282,G5020,S,182472,0,+40.6849947,-073.7738873,"POLYGON ((-73.77709 40.68820, -73.77662 40.688..."


In [ ]:
# Merging our dataframes together on Geo_ID tag
first_merge = gpd2019shp.merge(Education_Income, on=GEOID)
first_merge

In [17]:
#3 reading in our GeoJSON file for MTA_Subway_Stations linking GPS coordinates
url_for_MTA = "https://data.cityofnewyork.us/api/geospatial/arq3-7z49?method=export&format=GeoJSON"
MTA_Subway_Stations = gpd.read_file(url_for_MTA)

In [18]:
MTA_Subway_Stations

,name,url,line,objectid,notes,geometry
0,Astor Pl,http://web.mta.info/nyct/service/,4-6-6 Express,1,"4 nights, 6-all times, 6 Express-weekdays AM s...",POINT (-73.99107 40.73005)
1,Canal St,http://web.mta.info/nyct/service/,4-6-6 Express,2,"4 nights, 6-all times, 6 Express-weekdays AM s...",POINT (-74.00019 40.71880)
2,50th St,http://web.mta.info/nyct/service/,1-2,3,"1-all times, 2-nights",POINT (-73.98385 40.76173)
3,Bergen St,http://web.mta.info/nyct/service/,2-3-4,4,"4-nights, 3-all other times, 2-all times",POINT (-73.97500 40.68086)
4,Pennsylvania Ave,http://web.mta.info/nyct/service/,3-4,5,"4-nights, 3-all other times",POINT (-73.89489 40.66471)
...,...,...,...,...,...,...
468,Coney Island - Stillwell Av,http://web.mta.info/nyct/service/,D-F-N-Q,469,"D,F,N,Q-all times",POINT (-73.98124 40.57728)
469,34th St - Hudson Yards,http://web.mta.info/nyct/service/,7-7 Express,470,"7-all times, 7 Express-rush hours AM westbound...",POINT (-74.00220 40.75545)
470,72nd St,http://web.mta.info/nyct/service/,Q,641,Q-all times,POINT (-73.95836 40.76880)
471,86th St,http://web.mta.info/nyct/service/,Q,642,Q-all times,POINT (-73.95177 40.77786)


In [ ]:
#4 Merge GeoID tags after both dataframes columns have been cleaned to match


In [ ]:
#Spatial Join using geometry column in both gpd.dataframes, state "within" or 'without' 
sjoined_listings = gpd.sjoin(gdf_listings, stockholm_areas, op=”within”)
sjoined_listings.head()